# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# Confirm that our new database was created

databases = mongo.list_database_names()
print("Databases available:", databases)

Databases available: ['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# Assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# Review the collections in our new database

collections = db.list_collection_names()
print("Collections in 'uk_food':", collections)

Collections in 'uk_food': ['establishments']


In [6]:
# Review a document in the establishments collection

# Take one random document for review

first_document = db.establishments.find_one()
print("First document in 'establishments' collection:")
pprint(first_document)

First document in 'establishments' collection:
{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Coastguard Inn',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 1034540,
 'LocalAuthorityBusinessID': 'PI/000078691',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6622f9c5db9dae448a4f8db2'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1034540',
            'rel': 'self'}],
 'meta': {'dataSource

In [7]:
# Assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data

new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection

insert_result = establishments.insert_one(new_restaurant)
print("Inserted document ID:", insert_result.inserted_id)

Inserted document ID: 6622f9d165f2db7ae1cbf2fb


In [10]:
# Check that the new restaurant was inserted

search_result = establishments.find_one({"BusinessName": "Penang Flavours"})

# Print the result to verify insertion
if search_result:
    print("Restaurant found in the database:")
    pprint(search_result)
else:
    print("Restaurant not found in the database.")

Restaurant found in the database:
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6622f9d165f2db7ae1cbf2fb'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Execute the query and print the results
restaurant_type = establishments.find_one(query, projection)

print("Restaurant type:")
pprint(restaurant_type)

Restaurant type:
{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID

# Assign the found value to a variable
business_type_id = restaurant_type["BusinessTypeID"]

if business_type_id:
    update_result = establishments.update_one(
        {"BusinessName": "Penang Flavours"},
        {"$set": {"BusinessTypeID": business_type_id}}
    )
    print("Documents updated:", update_result.modified_count)
else:
    print("BusinessTypeID not found for 'Restaurant/Cafe/Canteen'")

Documents updated: 1


In [13]:
# Confirm that the new restaurant was updated

updated_document = establishments.find_one({"BusinessName": "Penang Flavours"})

# Print the result to verify the update
if updated_document:
    print("Updated document for 'Penang Flavours':")
    pprint(updated_document)
else:
    print("Restaurant 'Penang Flavours' not found in the database.")

Updated document for 'Penang Flavours':
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6622f9d165f2db7ae1cbf2fb'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"

dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of establishments in Dover:", dover_count)

Number of establishments in Dover: 994


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"

delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
print("Deleted documents count:", delete_result.deleted_count)

Deleted documents count: 994


In [16]:
# Check if any remaining documents include Dover

remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Remaining establishments in Dover:", remaining_dover_count)

Remaining establishments in Dover: 0


In [17]:
# Check that other documents remain with 'find_one'

sample_document = establishments.find_one()

# Print a sample document to verify that the collection still contains data
if sample_document:
    print("A sample document from the remaining establishments:")
    pprint(sample_document)
else:
    print("No documents remain in the collection.")

A sample document from the remaining establishments:
{'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.912144538092,
 'FHRSID': 289352,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6AB',
 'RatingDate': '2015-08-27T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6622f9c5db9dae448a4f9098'),
 'geocode': {'latitude': '51.08084', 'longitude': '1.188537'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/289352',
            'rel': 'self'}],
 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude

establishments.update_many({}, [
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}},
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}
])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [20]:
# Change the data type from String to Integer for RatingValue

establishments.update_many({}, [
    {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [21]:
# Check that the coordinates and rating value are now numbers
# Fetch some sample documents to check the data types of latitude, longitude, and RatingValue

sample_documents = establishments.find({}, {"geocode.latitude": 1, "geocode.longitude": 1, "RatingValue": 1, "_id": 0}).limit(5)

# Print the types of latitude, longitude, and RatingValue for verification
for doc in sample_documents:
    print("Extract from a sample document:")
    pprint(doc)
    print(f"Latitude: {doc['geocode']['latitude']} Type: {type(doc['geocode']['latitude'])}")
    print(f"Longitude: {doc['geocode']['longitude']} Type: {type(doc['geocode']['longitude'])}")
    print(f"RatingValue: {doc['RatingValue']} Type: {type(doc['RatingValue'])}")
    print("--------------------------------")

Extract from a sample document:
{'RatingValue': 5, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
Latitude: 51.08084 Type: <class 'float'>
Longitude: 1.188537 Type: <class 'float'>
RatingValue: 5 Type: <class 'int'>
--------------------------------
Extract from a sample document:
{'RatingValue': 5, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
Latitude: 51.08084 Type: <class 'float'>
Longitude: 1.188537 Type: <class 'float'>
RatingValue: 5 Type: <class 'int'>
--------------------------------
Extract from a sample document:
{'RatingValue': 5, 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
Latitude: 51.083812 Type: <class 'float'>
Longitude: 1.195625 Type: <class 'float'>
RatingValue: 5 Type: <class 'int'>
--------------------------------
Extract from a sample document:
{'RatingValue': 4, 'geocode': {'latitude': 51.08084, 'longitude': 1.188537}}
Latitude: 51.08084 Type: <class 'float'>
Longitude: 1.188537 Type: <class 'float'>
RatingValue: 4 Type: <clas